# Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torch.utils.data as data
import torchvision.transforms as transforms

# Constants definition

In [2]:
EPOCHS = 5
IMAGE_SIZE = 28
# Keras's default learning rate is 1e-3
LEARNING_RATE = 1e-3
# Keras's default batch size is 32
BATCH_SIZE = 32

# Providing the data

In [3]:
transform = transforms.ToTensor()
train_dataset = datasets.FashionMNIST('../data/fashionMNIST', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST('../data/fashionMNIST', train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Define and Compile the Neural Network

In [4]:
# CrossEntropyLoss in PyTorch assumes unnormalized values, thus Softmax should not be used
# https://stackoverflow.com/a/61438119
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(IMAGE_SIZE * IMAGE_SIZE, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss()

# Training the Neural Network

In [5]:
for epoch in range(EPOCHS):
    print(f'Epoch {epoch+1}/{EPOCHS}')
    loss_sum = 0
    correct = 0
    for images, labels in train_loader:
        optimizer.zero_grad()
        out = model(images)
        loss = loss_fn(out, labels)
        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(out.data, 1)
        correct += (predicted == labels).sum().item()
        loss_sum += loss.item()

    print(f'loss: {loss_sum / len(train_loader):e} - acc: {correct / len(train_dataset)}')

Epoch 1/5
loss: 5.168233e-01 - acc: 0.8178666666666666
Epoch 2/5
loss: 3.814406e-01 - acc: 0.8627333333333334
Epoch 3/5
loss: 3.408272e-01 - acc: 0.8759166666666667
Epoch 4/5
loss: 3.166639e-01 - acc: 0.8841666666666667
Epoch 5/5
loss: 2.988156e-01 - acc: 0.88985


# Evaluating the result

In [6]:
correct = 0
with torch.no_grad():
    for images, labels in test_loader:
        out = model(images)
        _, predicted = torch.max(out.data, 1)
        correct += (predicted == labels).sum().item()
print(f'Test accuracy: {100 * correct / len(test_dataset)}%')

Test accuracy: 86.78%
